#### 유형1

In [ ]:
import pandas as pd

df = pd.read_csv("data/mtcars.csv")
print(df)
# print(df.info())

# print(df['qsec'])

# 'qsec' 칼럼의 값을 0 ~ 1 사이의 값으로 변환(정규화)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df['qsec'] = scaler.fit_transform(df[['qsec']])
# print(df['qsec'])
value = df['qsec'] > 0.5
print(value)

# 개수
print(value.sum())

#### 유형2

In [ ]:
import pandas as pd

train = pd.read_csv("data/customer_train.csv")
test = pd.read_csv("data/customer_test.csv")

# print(train.info())
# print(test.info())

# print(train.isnull().sum())

# 결측치 대체 - 환불금액
mean = train['환불금액'].mean()
# print(median)
train['환불금액'] = train['환불금액'].fillna(mean)
test['환불금액'] = test['환불금액'].fillna(mean)

# 변수 선택
target = train.pop('성별')

# 타입 변환 - 주구매상품, 주구매지점
cols = ['주구매상품', '주구매지점']

from sklearn.preprocessing import LabelEncoder
# 인코딩
le = LabelEncoder()

for col in cols:
	train[col] = le.fit_transform(train[col])
	test[col] = le.fit_transform(test[col])
	
# print(train.info())

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
	train, target, test_size=0.2, random_state=3)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

model = RandomForestClassifier(max_depth=7, n_estimators=200, random_state=3)
model = RandomForestClassifier(random_state=3)
model.fit(X_train, y_train)
pred = model.predict(X_test)
# print(pred)

score = roc_auc_score(y_test, pred)
print("roc_auc:", score)

# test 데이터 측정
pred2 = model.predict(test)

submit = pd.DataFrame({'pred': pred2})
submit.to_csv('result.csv', index=False)

print(pd.read_csv('result.csv'))

#### 유형3

In [ ]:
# 1번
table = pd.crosstab(df['Gender'], df['Survived'])
# print(table)

from scipy import stats

result = stats.chi2_contingency(table)
# print(result)
print(round(result.statistic, 3))

# 2번
from statsmodels.formula.api import logit

formula = "Survived ~ Gender + SibSp + Parch + Fare"

model = logit(formula, data=df).fit()
print(model.summary())

parch = model.params['Parch']
print(round(parch, 3))

# 3번
import numpy as np

coef = model.params['SibSp']
# print(coef)
odds = np.exp(coef)
print(round(odds, 3))